# Run streamlit app from a Google Colab Notebook
> Created by [Manuel Romero](https://twitter.com/mrm8488)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q streamlit
!npm install localtunnel
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.876s

3 p

In [3]:
!cp /content/drive/MyDrive/new_data_aug.zip /content
!unzip /content/drive/MyDrive/new_data_aug.zip

Archive:  /content/drive/MyDrive/new_data_aug.zip
   creating: new_data_aug/test/
   creating: new_data_aug/test/images/
  inflating: new_data_aug/test/images/0-water.png  
  inflating: new_data_aug/test/images/1009-water.png  
  inflating: new_data_aug/test/images/1014-water.png  
  inflating: new_data_aug/test/images/1019-water.png  
  inflating: new_data_aug/test/images/1023-water.png  
  inflating: new_data_aug/test/images/102-water.png  
  inflating: new_data_aug/test/images/1041-water.png  
  inflating: new_data_aug/test/images/1043-water.png  
  inflating: new_data_aug/test/images/1063-water.png  
  inflating: new_data_aug/test/images/1068-water.png  
  inflating: new_data_aug/test/images/1075-water.png  
  inflating: new_data_aug/test/images/109-water.png  
  inflating: new_data_aug/test/images/1125-water.png  
  inflating: new_data_aug/test/images/1126-water.png  
  inflating: new_data_aug/test/images/114-water.png  
  inflating: new_data_aug/test/images/116-water.png  
  infl

In [63]:
!rm -rf /content/output_frames

In [64]:
import os
os.mkdir("/content/output_frames")

In [65]:
%cd /content/detectron2_repo

/content/detectron2_repo


## Create a streamlit app example


In [66]:
%%writefile app.py

import streamlit as st
import io
import cv2
import logging
from moviepy.editor import * 


import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, glob
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from PIL import Image
import torch
import natsort


st.markdown('<style>body{background-color: Blue;}</style>',unsafe_allow_html=True)
image_p = Image.open(r'/content/drive/MyDrive/yash.png')
#rgb_im_p = image_p.convert('RGB')
st.sidebar.image(image_p, use_column_width=True)

st.title("Water Segmentation Accelerator")

uploaded_file = st.file_uploader("Upload a video..", type=["mp4"])
temporary_location = False

if uploaded_file is not None:
    g = io.BytesIO(uploaded_file.read())  ## BytesIO Object
    temporary_location = "testout_simple.mp4"

    with open(temporary_location, 'wb') as out:  ## Open temporary file as bytes
        out.write(g.read())  ## Read bytes into file

    # close file
    out.close()


    video_file = open('/content/detectron2_repo/testout_simple.mp4', 'rb')
    video_bytes = video_file.read()
    st.video(video_bytes)

def my_fun(path):
  
  
  register_coco_instances("water_train_latest", {}, "/content/new_data_aug/train/images/train.json", "/content/new_data_aug/train/images")
  register_coco_instances("water_valid_latest", {}, "/content/new_data_aug/valid/images/valid.json", "/content/new_data_aug/valid/images")
  fruits_nuts_metadata_train = MetadataCatalog.get("water_train_latest")
  fruits_nuts_metadata_valid = MetadataCatalog.get("water_valid_latest")
  dataset_dicts_train = DatasetCatalog.get("water_train_latest")
  dataset_dicts_valid = DatasetCatalog.get("water_valid_latest")
  cfg = get_cfg()
  cfg.merge_from_file(
      "/content/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
  )
  cfg.DATASETS.TRAIN = ("water_train_latest",)
  cfg.DATASETS.TEST = ("water_valid_latest",)  # no metrics implemented for this dataset
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 0.002
  cfg.SOLVER.MAX_ITER = (
        10000
  )  # 300 iterations seems good enough, but you can certainly train longer
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
      128
  )  # faster, and good enough for this toy dataset
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)
  cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/model_final_new_data_aug.pth")
  # cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
  # cfg.DATASETS.TEST = ("water_test", )
  predictor = DefaultPredictor(cfg)



  cap = cv2.VideoCapture(path)
  # frame_width = 512
  # frame_height = 512
  # frame_size = (frame_width,frame_height)

  fps = int(cap.get(cv2.CAP_PROP_FPS))
  print(fps)
  frame_rate = cap.get(5)
  print("frame rate",frame_rate)
  logging.info("frame rate",frame_rate)
  save_interval = 1
  count = 0
  frame_count = 0
  while cap.isOpened():
    ret, frame = cap.read()
    if ret:
      frame_count += 1

      if frame_count % (fps * save_interval) == 0:
        count = count + 1
        logging.info("count",count)
        # frame = cv2.resize(frame,[512,512])
        outputs = predictor(frame)

        masks=outputs["instances"].pred_masks
      
        x = []
        for mask in masks:
              x.append((mask.sum()).item())
        print(x)
        v = Visualizer(frame[:, :, ::-1],metadata=fruits_nuts_metadata_valid,scale=1.2)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        output_frame = v.get_image()[:, :, ::-1]
        a=mask.sum() 
        img=output_frame.copy()
        output_frame = cv2.putText(img, text='Size of water mask: '+str(x), org=(50, 50), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.75, color=(0,0,0),thickness=1)
        
        cv2.imwrite('/content/output_frames/'+str(count)+'water.jpg',output_frame)
        
        frame_count = 0

  # Break the loop
    else:
      break

  cap.release()
  cv2.destroyAllWindows()

path_to_video= "/content/detectron2_repo/testout_simple.mp4"
inference = st.button("Perform Inference")

if inference:
  my_fun(path_to_video)
  output = st.button("Display Output")
  if output:
    dir_path = "/content/output_frames"

    filenames = os.listdir(dir_path)
    sorted_filenames = natsort.natsorted(filenames)
    logging.info("Sorted filenames",sorted_filenames)
    img_array = []
    filename = []
    for filename in sorted_filenames:
        filename = '/content/output_frames/'+filename
        print(filename)
        
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
        
    out = cv2.VideoWriter('/content/output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
    
    for i in range(len(img_array)):
        #print(i)
        out.write(img_array[i])
    out.release()  

    # Load the AVI video file 
    avi_file = VideoFileClip('/content/output_video.avi') 
    # Set the output file name and path 
    mp4_file_path = '/content/output_video_mp4.mp4' 
    # Write the video to the MP4 file 
    avi_file.write_videofile(mp4_file_path) 
    # Close the AVI file 
    avi_file.close()

  video_file = open(mp4_file_path, 'rb')
  logging.info("Output video available")
  video_bytes = video_file.read()
  
  st.video(video_bytes) 
  
       

Overwriting app.py


## Install localtunnel

## Run streamlit in background

In [67]:
# !streamlit run /content/detectron2_repo/app.py &>/content/logs.txt &
!streamlit run /content/detectron2_repo/app.py --logger.level=debug &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.628s
your url is: https://great-hotels-design-34-172-105-203.loca.lt
